# Flaky VS True Failures Classification Results 

## Input

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import csv
import re
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
import filecmp
from lxml import etree
from pathlib import Path
import base64
import unicodedata
from tqdm import tqdm
import glob
import statistics

## Inputs

In [2]:
input = 'Result'
projectNames = {"spring-boot":"spring-projects-spring-boot",
                "alluxio" : "Alluxio-alluxio",
                "http-request": "kevinsawicki-http-request",
                "hbase" : "apache-hbase",
                "ambari": "apache-ambari",
                "java-webSocket" : "tootallnate-java-websocket",
                "wildfly": "wildfly-wildfly",
                "okhttp":"square-okhttp",
                "hector" : "hector-client-hector",
                "wro4j": "wro4j-wro4j",
                "incubator-dubbo" : "apache-incubator-dubbo",
                "logback" : "qos-ch-logback",
                "activiti" : "activiti-activiti",
                "httpcore" : "apache-httpcore",
                "commons-exec" : "apache-commons-exec",
                "io-undertow" : "undertow-io-undertow",
                "orbit" : "orbit-orbit",
                "assertj-core" : "assertj-core",
                "achilles" : "doanduyhai-Achilles",
                "handlebars.java" : "handlebars.java",
                "elastic-job-lite" : "elasticjob-elastic-job-lite",
                "zxing" : "zxing-zxing",
                "ninja" : "ninja-ninja"}

output = 'Result'

In [3]:
DT = pd.read_csv('Result/DT_FinalClassifierResult.csv',index_col=False)
NB = pd.read_csv('Result/NB_FinalClassifierResult.csv',index_col=False)
tfidf = pd.read_csv('Result/TFIDF.csv',index_col=False)

In [4]:
targetResult = ['TP', 'FN','FP', 'TN', 'P', 'R', 'F1']
allResult = {}
for project in DT['Project'].unique():
    # if (project != "alluxio"):
        DT_balance = DT[(DT['Project']==project) & (DT['ResultTag']=='Balance+Duplicate+FailuresWithMutants')]
        DT_NotBalance = DT[(DT['Project']==project) & (DT['ResultTag']=='NotBalance+Duplicate+FailuresWithMutants')]
        NB_balance = NB[(NB['Project']==project) & (NB['ResultTag']=='Balance+Duplicate+FailuresWithMutants')]
        NB_NotBalance = NB[(NB['Project']==project) & (NB['ResultTag']=='NotBalance+Duplicate+FailuresWithMutants')]
        tfidfperProject = tfidf[tfidf['Project']==project]
        

        # iterate over the result
        dataset = {'DT|Balance':DT_balance,'DT|NotBalance':DT_NotBalance, 'NB|Balance':NB_balance, 'NB|NotBalance': NB_NotBalance,'tfidf|None':tfidfperProject}
        for k,d in dataset.items():
            if (len(d)>0):
                resultPerProject = []
                for c in targetResult:
                    resultPerProject.append(int(d[c].tolist()[0]))

                allResult[project+'|'+k] = '|'.join([str(i) for i in resultPerProject])


In [5]:
projects = [k.split('|')[0] for k in allResult.keys()]
finalResultAllProjects = []
useBalance = False
thold = 0.10
for p in set(projects):
    # if (p == 'alluxio'):
        perProjectDTBalance = {k:v for k,v in allResult.items() if k ==(p+'|DT|Balance')}
        perProjectDTNotBalance = {k:v for k,v in allResult.items() if k ==(p+'|DT|NotBalance')}
        perProjectTFIDF = {k:v for k,v in allResult.items() if k ==(p+'|tfidf|None')}
        
        

        perProjectResult = []
        perProjectResult.append(p)
        if (useBalance):
            ff = sum([int(x) for x in list(perProjectDTBalance.values())[0].split('|')][:2])
            tf = sum([int(x) for x in list(perProjectDTBalance.values())[0].split('|')][2:4])
            if (tf + ff >0):
                if (tf/ff < thold or ff/tf < thold):
                    perProjectResult.extend(list(perProjectDTBalance.values())[0].split('|') if len(perProjectDTBalance)>0 else [0,0,0,0,0,0,0])
                else:
                    perProjectResult.extend(list(perProjectDTNotBalance.values())[0].split('|') if len(perProjectDTNotBalance)>0 else [0,0,0,0,0,0,0])
                perProjectResult.extend(list(perProjectTFIDF.values())[0].split('|') if len(perProjectTFIDF)>0 else [0,0,0,0,0,0,0])
            else:
                perProjectResult.extend([0,0,0,0,0,0,0,0,0,0,0,0,0,0])
        else:
            perProjectResult.extend(list(perProjectDTNotBalance.values())[0].split('|') if len(perProjectDTNotBalance)>0 else [0,0,0,0,0,0,0])
            perProjectResult.extend(list(perProjectTFIDF.values())[0].split('|') if len(perProjectTFIDF)>0 else [0,0,0,0,0,0,0])
        
        # Add Total and total flaky and non true failures
        TotalFailures = sum([int(perProjectResult[a]) for a in [1,2,3,4]])
        OnlyFlaky = sum([int(perProjectResult[a]) for a in [1,2]])
        non_Flaky = sum([int(perProjectResult[a]) for a in [3,4]])

        perProjectResult.insert(1,OnlyFlaky)
        perProjectResult.insert(1,non_Flaky)
        perProjectResult.insert(1,TotalFailures)
        finalResultAllProjects.append(perProjectResult)

allProject_df = pd.DataFrame(finalResultAllProjects,columns=['Project','TotalFailures','TotalNonFlaky','TotalFlaky','TP_FailureLog','FN__FailureLog','FP__FailureLog','TN_FailureLog','P_FailureLog','R_FailureLog','F_FailureLog','TP_TFIDF','FN_TFIDF','FP_TFIDF','TN_TFIDF','P_TFIDF','R_TFIDF','F1_TFIDF'])


In [6]:
# order by the total number of tests
resultTests = pd.read_csv(output+'/TextBasedMatchingResult.csv',index_col=False)

projectsTests = resultTests[['Project','Test']]
projectsTests = projectsTests.dropna()
merged_df = pd.merge(allProject_df, projectsTests, on='Project', how='outer')

merged_df = merged_df.sort_values(by=['Test'], ascending=False).reset_index()
merged_df = merged_df[['Project','Test','TotalFailures','TotalFlaky','TotalNonFlaky','TP_FailureLog','FN__FailureLog','FP__FailureLog','TN_FailureLog','P_FailureLog','R_FailureLog','F_FailureLog','TP_TFIDF','FN_TFIDF','FP_TFIDF','TN_TFIDF','P_TFIDF','R_TFIDF','F1_TFIDF']]

# convert to int tp get the sum then
columnList = ['Test','TotalFailures','TotalFlaky','TotalNonFlaky','TP_FailureLog','FN__FailureLog','FP__FailureLog','TN_FailureLog','P_FailureLog','R_FailureLog','F_FailureLog','TP_TFIDF','FN_TFIDF','FP_TFIDF','TN_TFIDF','P_TFIDF','R_TFIDF','F1_TFIDF']
for c in columnList:
    merged_df[c] = merged_df[c].astype(int)

merged_df = merged_df[merged_df['TotalFailures']>0]
sum_row = merged_df[columnList].sum().rename('Total')
merged_df = merged_df.append(sum_row, ignore_index=True)


merged_df.to_csv(output+'/FailureLog_VS_TFIDF.csv',index=False)
